In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime 
import os

import tensorflow as tf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# temp

In [7]:
shape = (1,1)
shape
len(shape)

2

In [38]:
b = tf.sparse.to_dense(a)
b = tf.cast(b, tf.float32)


In [39]:
get_embedding2 = tf.matmul(b,embedding)
get_embedding2

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.21     , 0.41     , 0.51     , 0.11     ],
       [1.18     , 2.1799998, 2.68     , 0.68     ]], dtype=float32)>

# 基础

## Tensor

### constant tensor

张量的数据类型和numpy.array基本一一对应

In [3]:
i = tf.constant(1) # tf.int32 类型常量
l = tf.constant(1,dtype = tf.int64) # tf.int64 类型常量
f = tf.constant(1.23) #tf.float32 类型常量
d = tf.constant(3.14,dtype = tf.double) # tf.double 类型常量
s = tf.constant("hello world") # tf.string类型常量
b = tf.constant(True) #tf.bool类型常量


print(tf.int64 == np.int64) 
print(tf.bool == np.bool)
print(tf.double == np.float64)
print(tf.string == np.unicode) # tf.string类型和np.unicode类型不等价

True
True
True
False


### variable tensor

区别：
- 常量值不可以改变，常量的重新赋值相当于创造新的内存空间;变量的值可以改变，可以通过assign, assign_add等方法给变量重新赋值
- 变量能够被 TensorFlow 的自动求导机制所求导，常量不行

In [5]:
c = tf.constant([1.0,2.0])
print(c)
print(id(c))
c = c + tf.constant([1.0,1.0])
print(c)
print(id(c))
print('--------')
v = tf.Variable([1.0,2.0],name = "v")
print(v)
print(id(v))
v.assign_add([1.0,1.0])
print(v)
print(id(v))

tf.Tensor([1. 2.], shape=(2,), dtype=float32)
140338867020240
tf.Tensor([2. 3.], shape=(2,), dtype=float32)
140338867020432
--------
<tf.Variable 'v:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
140338870593552


<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>

<tf.Variable 'v:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
140338870593552


## dataset
Dataset数据结构应用非常灵活，因为它本质上是一个Sequece序列，其每个元素可以是各种类型，例如可以是张量，列表，字典，也可以是Dataset。

In [5]:
from sklearn import datasets 
iris = datasets.load_iris()
dfiris = pd.DataFrame(iris["data"],columns = iris.feature_names)

In [25]:
ds2 = tf.data.Dataset.from_tensor_slices((dfiris.to_dict('list'),iris['target']))
for features,label in ds2.take(1):
    type(features)
    list(features.keys())[0]
    list(features.values())[0].numpy()
    print('-------------')
    print(features, label)

dict

'sepal length (cm)'

5.1

-------------
{'sepal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=5.1>, 'sepal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=3.5>, 'petal length (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=1.4>, 'petal width (cm)': <tf.Tensor: shape=(), dtype=float32, numpy=0.2>} tf.Tensor(0, shape=(), dtype=int64)


## layer

### 自定义layer

需要重写init、build、call方法。build方法主要是为了当你用到layer的时候，自适应input_shape。也可以写在init里，但是这样需要提前指定input_shape。很多时候我们会在model里调用layer，并不知道具体使用的时候input_shape是多少，所以建议还是写在build里

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    

x = tf.ones((2, 2))
# At instantiation, we don't know on what inputs this is going to get called
linear_layer = Linear(32)

# The layer's weights are created dynamically the first time the layer is called
y = linear_layer(x)


# 模型

## 构建

In [2]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(16,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
# Now the model will take as input arrays of shape (None, 16)
# and output arrays of shape (None, 32).
# Note that after the first layer, you don't need to specify
# the size of the input anymore:
model.add(tf.keras.layers.Dense(32))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
Total params: 1,600
Trainable params: 1,600
Non-trainable params: 0
_________________________________________________________________


In [4]:
model